In [1]:
from io import BytesIO
import pandas as pd
import datetime
import vk_api
import os
import requests
import json
import random
import numpy as np
import plotly.express as px
import os

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import sys

In [2]:
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials

In [3]:
df_customersid = pd.read_csv("/mnt/HC_Volume_18315164/home-jupyter/jupyter-g-zigora-23/first_project/olist_customers_dataset.csv")

In [4]:
df_customersid.head(5)

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [5]:
df_orders = pd.read_csv("/mnt/HC_Volume_18315164/home-jupyter/jupyter-g-zigora-23/first_project/olist_orders_dataset.csv")

In [6]:
df_orders.head(5)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [7]:
df_order_items = pd.read_csv("/mnt/HC_Volume_18315164/home-jupyter/jupyter-g-zigora-23/first_project/olist_order_items_dataset.csv")

In [8]:
df_order_items.head(5)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


###  1. Сколько у нас пользователей, которые совершили покупку только один раз?


In [9]:
df = df_orders.merge(df_customersid, on="customer_id")

In [10]:
df_delivered = df[df.order_status == 'delivered']

In [11]:
orders_byuser = df_delivered.groupby("customer_unique_id", as_index=False).count()

In [12]:
orders_byuser[orders_byuser.order_id == 1]

,customer_unique_id,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_zip_code_prefix,customer_city,customer_state
0,0000366f3b9a7992bf8c76cfdf3221e2,1,1,1,1,1,1,1,1,1,1,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1,1,1,1,1,1,1,1,1,1,1
2,0000f46a3911fa3c0805444483337064,1,1,1,1,1,1,1,1,1,1,1
3,0000f6ccb0745a6a4b88665a16c9f078,1,1,1,1,1,1,1,1,1,1,1
4,0004aac84e0df4da2b147fca70cf8255,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
93353,fffcf5a5ff07b0908bd4e2dbc735a684,1,1,1,1,1,1,1,1,1,1,1
93354,fffea47cd6d3cc0a88bd621562a9d061,1,1,1,1,1,1,1,1,1,1,1
93355,ffff371b4d645b6ecea244b27531430a,1,1,1,1,1,1,1,1,1,1,1
93356,ffff5962728ec6157033ef9805bacc48,1,1,1,1,1,1,1,1,1,1,1


### Мною были предприняты методы группировки, выявлено кол-во уникальных покупателей, затем отфильтрованы покупатели с 1 покупкой

### 2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)? (5 баллов)



### Здесь я перевел как и в прошлой задаче столбик в datetime

In [13]:
df_orders.order_purchase_timestamp = pd.to_datetime(df_orders.order_purchase_timestamp)

### Затем выявил недоступные или  
### отмененные заказы через или ( | )

In [14]:
df_ordersfail = df_orders[(df_orders.order_status == "unavailable") | (df_orders.order_status == "canceled")] 

### Вывели месяцы

In [15]:
df_orders.order_purchase_timestamp.dt.month

0        10
1         7
2         8
3        11
4         2
         ..
99436     3
99437     2
99438     8
99439     1
99440     3
Name: order_purchase_timestamp, Length: 99441, dtype: int64

### Сгруппировал месяц год статус 

In [16]:
df_ordersfaildbymonth = df_ordersfail.groupby([df_ordersfail.order_purchase_timestamp.dt.month, df_ordersfail.order_purchase_timestamp.dt.year, df_ordersfail.order_status], as_index=False).count()

In [17]:
df_ordersfail.groupby([df_ordersfail.order_purchase_timestamp.dt.year, df_ordersfail.order_purchase_timestamp.dt.month, df_ordersfail.order_status]).count()

order_id  \
order_purchase_timestamp order_purchase_timestamp order_status             
2016                     9                        canceled             2   
                         10                       canceled            24   
                                                  unavailable          7   
2017                     1                        canceled             3   
                                                  unavailable         10   
                         2                        canceled            17   
                                                  unavailable         45   
                         3                        canceled            33   
                                                  unavailable         32   
                         4                        canceled            18   
                                                  unavailable          9   
                         5                        canceled            29   
                                                  unavailable         31   
                         6                        canceled            16   
                                                  unavailable         24   
                         7                        canceled            28   
                                                  unavailable         52   
                         8                        canceled            27   
                                                  unavailable         32   
                         9                        canceled            20   
                                                  unavailable         38   
                         10                       canceled            26   
                                                  unavailable         58   
                         11                       canceled            37   
                                                  unavailable         84   
                         12                       canceled            11   
                                                  unavailable         42   
2018                     1                        canceled            34   
                                                  unavailable         48   
                         2                        canceled            73   
                                                  unavailable         30   
                         3                        canceled            26   
                                                  unavailable         17   
                         4                        canceled            15   
                                                  unavailable          5   
                         5                        canceled            24   
                                                  unavailable         16   
                         6                        canceled            18   
                                                  unavailable          4   
                         7                        canceled            41   
                                                  unavailable         18   
                         8                        canceled            84   
                                                  unavailable          7   
                         9                        canceled            15   
                         10                       canceled             4   

                                                                customer_id  \
order_purchase_timestamp order_purchase_timestamp order_status                
2016                     9                        canceled                2   
                         10                       canceled               24   
                                                  unavailable             7   
2017                     1                        canceled                3   
                 

In [18]:
df_ordersfaildbymonth.head()

,order_status,order_id,customer_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,canceled,3,3,3,2,0,0,3
1,unavailable,10,10,10,10,0,0,10
2,canceled,34,34,34,33,26,0,34
3,unavailable,48,48,48,48,0,0,48
4,canceled,17,17,17,15,0,0,17


### Взял среднюю, для визуального удобства сгруппировал по статусу

In [19]:
df_ordersfaildbymonth.groupby("order_status").mean()

,order_id,customer_id,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
order_status,,,,,,,
canceled,26.041667,26.041667,26.041667,20.166667,3.125,0.25,26.041667
unavailable,29.000000,29.000000,29.000000,29.000000,0.000,0.00,29.000000


### 3. По каждому товару определить, в какой день недели товар чаще всего покупается. (5 баллов)

### Мерджим, создаем колонку "weekday" чтобы по дням получить сортировку

In [20]:
df2 = df_orders.merge(df_order_items, on="order_id")

In [21]:
df2["weekday"] = df2.order_purchase_timestamp.dt.weekday

### Группируем айди и колонку с днями

In [22]:
df2_itemday = df2.groupby([df2.product_id, df2.weekday], as_index=False).count()

### Сортируем по айди заказа и айди продукта и берем с первого дня

In [23]:
df2_itemday = df2_itemday.sort_values("order_id", ascending=False).groupby(df2_itemday.product_id).first()

In [24]:
df2_itemday.groupby("weekday").count()

,product_id,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,seller_id,shipping_limit_date,price,freight_value
weekday,,,,,,,,,,,,,,
0,6133,6133,6133,6133,6133,6133,6133,6133,6133,6133,6133,6133,6133,6133
1,5620,5620,5620,5620,5620,5620,5620,5620,5620,5620,5620,5620,5620,5620
2,5233,5233,5233,5233,5233,5233,5233,5233,5233,5233,5233,5233,5233,5233
3,4686,4686,4686,4686,4686,4686,4686,4686,4686,4686,4686,4686,4686,4686
4,4433,4433,4433,4433,4433,4433,4433,4433,4433,4433,4433,4433,4433,4433
5,3252,3252,3252,3252,3252,3252,3252,3252,3252,3252,3252,3252,3252,3252
6,3594,3594,3594,3594,3594,3594,3594,3594,3594,3594,3594,3594,3594,3594


### 4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? Не стоит забывать, что внутри месяца может быть не целое количество недель. Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть. (8 баллов) 

In [25]:
df_merge4 = df_orders.merge(df_customersid, on="customer_id")

In [26]:
df_merge4.order_purchase_timestamp = pd.to_datetime(df_merge4.order_purchase_timestamp)       

In [27]:
df_merge4.order_purchase_timestamp.dt.week

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """Entry point for launching an IPython kernel.


0        40
1        30
2        32
3        46
4         7
         ..
99436    10
99437     6
99438    34
99439     2
99440    10
Name: order_purchase_timestamp, Length: 99441, dtype: int64

### Группируем по айди товара, по году и недели

In [28]:
df_iditem = df_merge4.groupby([df_merge4.customer_unique_id, df_merge4.order_purchase_timestamp.dt.week, df_merge4.order_purchase_timestamp.dt.year], as_index=False).count()

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  """Entry point for launching an IPython kernel.


### max() и min() для проверки 

In [29]:
df_iditem.groupby("customer_unique_id").mean()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_zip_code_prefix,customer_city,customer_state
customer_unique_id,,,,,,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
0000b849f77a49e4a4ce2b2a4ca5be3f,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
0000f46a3911fa3c0805444483337064,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
0000f6ccb0745a6a4b88665a16c9f078,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
0004aac84e0df4da2b147fca70cf8255,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
fffea47cd6d3cc0a88bd621562a9d061,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
ffff371b4d645b6ecea244b27531430a,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


### 5. Используя pandas, проведи когортный анализ пользователей. В период с января по декабрь выяви когорту с самым высоким retention на 3й месяц. Описание подхода можно найти тут. (10 баллов)

In [30]:
df5 = df_orders.merge(df_customersid, on="customer_id")

In [31]:
df5.order_purchase_timestamp = pd.to_datetime(df5.order_purchase_timestamp)

In [32]:
df5["month"] = df5.order_purchase_timestamp.dt.month

### Нужно найти первый заказ у каждого покупателя, и группируем по уникальным

In [33]:
first_orders = df5[["customer_unique_id", "order_purchase_timestamp"]].groupby("customer_unique_id", as_index=False).min()

### Импортируем расширенную версию datetime чтобы суммировать 3 месяца условные и узнать покупал ли клиент в это время второй раз товар, retention 3мес

In [34]:
from dateutil.relativedelta import relativedelta

In [35]:
first_orders["three_month"] = first_orders.order_purchase_timestamp.apply(lambda x: x + relativedelta(months = 3))

In [36]:
first_orders["two_month"] = first_orders.order_purchase_timestamp.apply(lambda x: x + relativedelta(months = 2))

In [37]:
df_all = df5.merge(first_orders, on="customer_unique_id")

### Сравнение соответственно
### Получили тех клиентов которые вернулись и купили второй раз, те что нам нужны

In [38]:
df_returned = df_all[(df_all.order_purchase_timestamp_x >= df_all.two_month) & (df_all.order_purchase_timestamp_x <= df_all.three_month)].customer_unique_id

In [39]:
df_returned

64       51838d41add414a0b1b989b7d251d9ee
269      ba77e9b6506636dcbd03e463d4786f24
270      ba77e9b6506636dcbd03e463d4786f24
282      ecc6d0f3eb9435df737e1f0d9a1c6d3f
437      95cfb39a7e263160308a934e42cfc4e0
                       ...               
79929    0aa1ab39ff2b837c05d7c9ff85b6d983
82593    dca9a13536adcef18c6c5859487347b1
83331    dad9e634284872d012677965e512b57d
84749    110991b379efbcebb0d35cbd20bf9698
89176    27d59fe7ff2ed894f826fd4a7e5466ea
Name: customer_unique_id, Length: 259, dtype: object

In [40]:
first_orders["month"] = first_orders.order_purchase_timestamp.dt.month

In [41]:
df_firstordermonth = first_orders.groupby("month").count().customer_unique_id

In [42]:
df_firstordermonth

month
1      7789
2      8203
3      9601
4      9063
5     10218
6      9079
7      9965
8     10455
9      4139
10     4792
11     7304
12     5488
Name: customer_unique_id, dtype: int64

In [43]:
df_repeatordermonth = first_orders[first_orders.customer_unique_id.isin(df_returned)].groupby("month").count().customer_unique_id

In [44]:
df_repeatordermonth / df_firstordermonth

month
1     0.003081
2     0.003170
3     0.002916
4     0.001986
5     0.003621
6     0.002423
7     0.001004
8     0.001052
9     0.004590
10    0.000835
11    0.002875
12    0.003280
Name: customer_unique_id, dtype: float64

### Retention невысокий, треубется проработка плана по удержанию клиента

### 6. Часто для качественного анализа аудитории использую подходы, основанные на сегментации. Используя python, построй RFM-сегментацию пользователей, чтобы качественно оценить свою аудиторию. В кластеризации можешь выбрать следующие метрики: R - время от последней покупки пользователя до текущей даты, F - суммарное количество покупок у пользователя за всё время, M - сумма покупок за всё время. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров. Пример такого описания: RFM-сегмент 132 (recency=1, frequency=3, monetary=2) имеет границы метрик recency от 130 до 500 дней, frequency от 2 до 5 заказов в неделю, monetary от 1780 до 3560 рублей в неделю. Описание подхода можно найти тут. (35 баллов)

In [45]:
df6 = df_orders.merge(df_customersid, on="customer_id")

In [46]:
df6.order_purchase_timestamp = pd.to_datetime(df6.order_purchase_timestamp)

In [47]:
df6["month"] = df6.order_purchase_timestamp.dt.month

In [48]:
first_orders_d6 = df6[["customer_unique_id", "order_purchase_timestamp"]].groupby("customer_unique_id", as_index=False).min()

In [49]:
df_ordersgood = df6[(df6.order_status == "delivered")]

In [50]:
df_ordersbymonth = df_ordersgood.groupby([df_ordersgood.order_purchase_timestamp.dt.month, df_ordersgood.order_purchase_timestamp.dt.year, df_ordersgood.order_status], as_index=False).count()

In [51]:
df_ordersgood.groupby([df_ordersgood.order_purchase_timestamp.dt.year, df_ordersgood.order_purchase_timestamp.dt.month, df_ordersgood.order_status]).count()

order_id  \
order_purchase_timestamp order_purchase_timestamp order_status             
2016                     9                        delivered            1   
                         10                       delivered          265   
                         12                       delivered            1   
2017                     1                        delivered          750   
                         2                        delivered         1653   
                         3                        delivered         2546   
                         4                        delivered         2303   
                         5                        delivered         3546   
                         6                        delivered         3135   
                         7                        delivered         3872   
                         8                        delivered         4193   
                         9                        delivered         4150   
                         10                       delivered         4478   
                         11                       delivered         7289   
                         12                       delivered         5513   
2018                     1                        delivered         7069   
                         2                        delivered         6555   
                         3                        delivered         7003   
                         4                        delivered         6798   
                         5                        delivered         6749   
                         6                        delivered         6099   
                         7                        delivered         6159   
                         8                        delivered         6351   

                                                                customer_id  \
order_purchase_timestamp order_purchase_timestamp order_status                
2016                     9                        delivered               1   
                         10                       delivered             265   
                         12                       delivered               1   
2017                     1                        delivered             750   
                         2                        delivered            1653   
                         3                        delivered            2546   
                         4                        delivered            2303   
                         5                        delivered            3546   
                         6                        delivered            3135   
                         7                        delivered            3872   
                         8                        delivered            4193   
                         9                        delivered            4150   
                         10                       delivered            4478   
                         11                       delivered            7289   
                         12                       delivered            5513   
2018                     1                        delivered            7069   
                         2                        delivered            6555   
                         3                        delivered            7003   
                         4                        delivered            6798   
                         5                        delivered            6749   
                         6                        delivered            6099   
                         7                        delivered            6159   
                         8                        delivered            6351   

                                                                order_purchase_timestamp  \
order_purchase_timestamp order_purchase_timestamp order_status                             
201